# Inlämningsuppgift2 - Steam Sale!
Mattias Kallman 2019


## Uppgiftens beskrivning:

Uppgiften handlar om att lyckas scrapea data från steams butiksida för produkter med nedsatta pris. Data ska rensas och det relevanta ska appendas till en .csv fil. Tanken är att man kör scriptet t.ex en gång i veckan för att lägga till data till excelfilen.
Man kunde även exportera filen som ett .py script från jupyter notebook och automatisera körandet av filen, men det här hör inte till den här kursen.

Bekanta dig med Steams produktsida (specials) ​HÄR
(öppna källkoden, kolla t.ex vilka css klasser som kunde användas mm.)
### 1. Data Scraping och Data Munging
**ok** - Ladda ner sida 1 av erbjudanden på steam. Skapa en tabell med följande rubriker och fyll tabellen med data från sajten.

**ok** - **Spelnamn|Rating|#Reviews|Rabatt%|Pris|OrdinariePris|Utgivningsår|Win|Lin|OSX|Tid**

**ok** - Skapa en sifferbaserad rating enligt alternativen som steam använder i textform (Mostly Positive, Overwhelmingly Positive ...).

**ok** - Använd 0 eller 1 för att beteckna ifall spelet stöder platformen (operativsystem).

**ok** - Kom ihåg att granska (och ge mig en bild av datan) genom att använda t.ex. head() och describe() för att få en bättre översikt över den insamlade datan.

**ok** - För fulla poäng krävs att man samlar data från 5 sidor med erbjudanden.


**ok** - Skapa en CSV fil (men endast ifall den inte redan finns!) och skriv in datan som har samlats. På så vis kan scriptet skapa en fil ifall den inte finns, och ifall den finns fortsätter datainsamlingen.


### 2. Reflektion och feedback
**ok** - Kommentera ditt utförande och visa eventuella delmoment du utförde som kommentarer i din .ipynb fil. 

**ok** - Använd markup för att formattera längre förklaringar över vad du gjort i textform ovanför kodrutan, och kommentarer (#) i din kod för att illustrera vilka variabler som innehåller vad för data i vilket skede.

### Importerar biblioteken jag kommer att behöva

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import pandas as pd
import numpy as np
import re
from datetime import datetime
import os

### Skapar en funktion som hämtar websidan för steam sales.

In [ ]:
# Function that fetches website and returns it as a parsed BeautifulSoup object
def get_website(website):
    return soup(urlopen(website).read())


In [ ]:
# Call function and assign values to a variable to work with locally.
web_page = get_website(website='https://store.steampowered.com/search/?specials=1')

In [ ]:
# Test that web page loads
print(web_page.prettify())

### Nedan skapar jag en funktion som konverterar  Steams betyg till sifferskala.

In [ ]:
# Function that converts Steam sale 'Strings' ratings into a numeric value. Returns 'n/a' if no match is found.
def convert_steam_rating_to_numeric_value(rating):
    if(rating == "Overwhelmingly Positive"):
        return 8
    elif(rating == "Very Positive"):
        return 7
    elif(rating == "Positive"):
        return 6
    elif(rating == "Mostly Positive"):
        return 5
    elif(rating == "Mixed"):
        return 4
    elif(rating == "Mostly Negative"):
        return 3
    elif(rating == "Negative"):
        return 2
    elif(rating == "Very Negative"):
        return 1
    elif(rating == "Overwhelmingly Negative"):
        return 0
    else:
        "n/a"

### Huvudfunktionen som gör det tunga lyftandet och hämtar datan, anpassar datan och returnerar den tillsist som en Matris.

In [ ]:
# Searches web page for list top 25 games, and scrapes for name, rating, number of ratings, discount %, discount price,
# original price, release date, os(true/false for: win, linux, mac) 
def get_games(soup_web_page):
    # Get current time
    now = datetime.now()
    # Saves the web page's form containing the list(<a>) of the games.
    games_list = soup_web_page.select_one(
        "form[id=advsearchform]").find_all("a",{"class":"search_result_row ds_collapse_flag"})
    
    # Variable where we save the treated data that will be returned at the end of the function.
    new_games_list = []
    
    # Loop thru all games from game_list and fetch data.
    for game in games_list:
        name = game.select_one("span[class=title]").text
        try:
            rating = game.find("span", {"class":"search_review_summary"})['data-tooltip-html']
            rating = rating.split("<br>")
            rating = rating[0]
        except:
            continue
        rating_score =convert_steam_rating_to_numeric_value(rating=rating)
        try:
            rating_numbers = game.find("span", {"class":"search_review_summary"})['data-tooltip-html']
            rating_numbers = rating_numbers.split("<br>")
            rating_numbers = rating_numbers[1].split(" ")
            rating_numbers = rating_numbers[3]
        except:
            continue
        try:
            discount_procent = game.find("div",{"class":"col search_discount responsive_secondrow"}).text
            discount_procent = discount_procent.replace("\n","")
        except:
            continue
        try:
            discount_price = game.find("div", {"class":"col search_price discounted responsive_secondrow"}).text
            discount_price = discount_price.split("€")
            discount_price = discount_price[1]
        except:
            continue         
        original_price = game.find("strike").text.replace("€","")
        year = game.find("div",{"class":"col search_released responsive_secondrow"}).text
        os = game.select_one("p")
        win = bool(re.search("win", str(os)))
        lin = bool(re.search("linux", str(os)))
        mac = bool(re.search("mac", str(os)))
        scrape_time = now
        
        # Once all data is collected for this iteration, the local variables are then appended to new_games_list.
        new_games_list.append(
            [name, rating, rating_score, rating_numbers, discount_procent, 
             discount_price, original_price, year, win, lin, mac, scrape_time])
        
    # Once data aggregation is done it is finally returned
    return new_games_list
    

In [ ]:
# Creating matrix with game data from web_page data
games_matrix = get_games(soup_web_page=web_page)

In [ ]:
# Function that converts normal python matrix to Pandas DataFrame model. Function also adds columns, etc.
def matrix_to_DataFrame(matrix):
    df = pd.DataFrame(matrix)
    df.columns = ['Names', 'Rating', 'Rating Score', 'Number of Ratings', 'Discount %', 'Discount Price', 'Normal Price', 'Release Date', 'Win', 'Linux', 'Mac', 'Scrape Time']
    return df

In [ ]:
# Creating DataFrame of games_matrix
df_steam_sale = matrix_to_DataFrame(matrix=games_matrix)

In [ ]:
df_steam_sale

## Spara till CSV filformat

In [ ]:
# Function that checks if file exists and appends data to .csv file, if not it creates .csv file with data.
def write_to_csv(data_frame):
    if not os.path.isfile(r'steamSale.csv'):
        data_frame.to_csv(r'steamSale.csv', header='column_names', index=False)
    else:
        data_frame.to_csv(r'steamSale.csv', mode='a', header=False, index=False)

In [ ]:
write_to_csv(data_frame=df_steam_sale)

In [ ]:
pd.read_csv('steamSale.csv')

# Till sist så samlar vi alla funktioner i en mästarfunktion som kör allt och sparar datan till en csv

In [ ]:
# Compiling all the functions together in one master function to scrape, munge and write to csv format.
def scraping_munging_writing_to_csv(url):
    web_page = get_website(website=url)
    games_matrix = get_games(soup_web_page=web_page)
    df_steam_sale = matrix_to_DataFrame(matrix=games_matrix)
    write_to_csv(data_frame=df_steam_sale)
    

#### Kör funktionen 5 ggr för att få 5 sidor av data med i csv:n.

In [ ]:
# Run the function 5 times with the 5 first pages from the steam sale site.
scraping_munging_writing_to_csv(url='https://store.steampowered.com/search/?specials=1')
scraping_munging_writing_to_csv(url='https://store.steampowered.com/search/?specials=1&page=2')
scraping_munging_writing_to_csv(url='https://store.steampowered.com/search/?specials=1&page=3')
scraping_munging_writing_to_csv(url='https://store.steampowered.com/search/?specials=1&page=4')
scraping_munging_writing_to_csv(url='https://store.steampowered.com/search/?specials=1&page=5')


In [ ]:
# Reads the csv file and displays the content
pd.read_csv('steamSale.csv')

# By Mattias Kallman 2019